##Problem 1:  generate text embeddings using any Hugging Face embedding model.

In [3]:
!pip install -qU  langchain langchain-huggingface sentence_transformers

In [4]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
text = "Testing the hugging face embedding model. "
query_result = embeddings.embed_query(text)
print(query_result)

[0.0020240808371454477, -0.04313012212514877, -0.027377011254429817, 0.0180827584117651, 0.01447655726224184, 0.04143153503537178, 0.04492202401161194, 0.03238696604967117, 0.013131218031048775, 0.02475319802761078, -0.011981315910816193, -0.07174444943666458, -0.004653293173760176, 0.0576627179980278, 0.08096615970134735, -0.05585964024066925, -0.004869895521551371, 0.02986510843038559, -0.014092854224145412, -0.026375822722911835, 0.04895314201712608, -0.008880414068698883, 0.024532591924071312, 0.0028689149767160416, -0.07352107018232346, 0.003868622938171029, -0.037432946264743805, 0.0437597930431366, -0.00850363727658987, -3.527813169057481e-05, -0.047471288591623306, 0.02712423726916313, 0.04406862333416939, 0.008906628005206585, 1.6871766774784192e-06, -0.029496634379029274, 0.0021003978326916695, -0.012181627564132214, 0.028616422787308693, 0.01189976092427969, 0.044796548783779144, -0.06378413736820221, 0.014167902991175652, -0.031113553792238235, -0.010620888322591782, -0.000

In [10]:
len(query_result)

768

##Problem 2: Perform similarity search using Langchain Pinecone Vector Store

In [11]:
!pip install -qU langchain langchain-pinecone langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompati

In [12]:
from google.colab import userdata

In [13]:
PINECONE_API_KEY=userdata.get("PINECONE_API_KEY")

In [15]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)


In [35]:
from pinecone import ServerlessSpec
index_name = "rag-demo-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=3072,  # must match embedding model
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)


Creating Embedding model

In [29]:
!pip install -qU google-genai
!pip install -qU langchain-google-genai

In [36]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
GEMINI_API_KEY=userdata.get("GEMINI_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(
    api_key=GEMINI_API_KEY,
    model="gemini-embedding-001"
)

Loading the documents

In [23]:
from langchain_core.documents import Document

doc1 = Document(page_content="Shah Rukh Khan, known as the King of Bollywood, is celebrated for his romantic roles and strong screen presence. He has starred in numerous iconic films like Dilwale Dulhania Le Jayenge and Chennai Express.")
doc2 = Document(page_content="Aamir Khan is known for his perfectionism and thought-provoking films. He has delivered memorable performances in movies like Lagaan, Dangal, and Taare Zameen Par, often focusing on social issues.")
doc3 = Document(page_content="Salman Khan is famous for his action-packed roles and mass appeal. Known for movies like Bajrangi Bhaijaan and Dabangg, he also hosts the popular reality show Bigg Boss.")
doc4 = Document(page_content="Ranbir Kapoor is admired for his versatility and charm. With films like Rockstar, Barfi, and Sanju, he has earned critical acclaim and a huge fan following among the youth.")
doc5 = Document(page_content="Deepika Padukone is one of the most successful actresses in Bollywood. Known for her powerful performances in films like Padmaavat and Piku, she has also worked in international projects.")

docs = [doc1, doc2, doc3, doc4, doc5]

Creating vector store

In [37]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [39]:
vector_store.add_documents(docs)

['54d9c7c4-536c-4375-8087-276d364d21e3',
 '7785f3b0-71fd-4799-8175-f027dcca6501',
 '75f851b5-c744-4aae-b28e-e1c27c0bd7c5',
 '8db1e468-2115-4113-bf0a-19abb29d94a0',
 '3a370baf-9557-48f0-ab1a-bf332012b207']

In [43]:
results = vector_store.similarity_search(
    "Which actor is known for fighting movies",
    k=2
)

for doc in results:
    print(doc.page_content)
    print()




Salman Khan is famous for his action-packed roles and mass appeal. Known for movies like Bajrangi Bhaijaan and Dabangg, he also hosts the popular reality show Bigg Boss.

Aamir Khan is known for his perfectionism and thought-provoking films. He has delivered memorable performances in movies like Lagaan, Dangal, and Taare Zameen Par, often focusing on social issues.

